In [ ]:
# @title Generate Matrix

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def generate_matrix(n):   # generates a random nxn matrix
  A = np.random.uniform(-5, 5, [n,n])   # entries range from -5 to 5
  return A

#A = np.array([[-2., 1.], [1., -2.]])   # manual option

A = generate_matrix(2)
print(f"A = \n {A}\n")

# calculate trace and determinant
Tr_A = np.trace(A)
Det_A = np.linalg.det(A)
print(f"Tr(A) = {Tr_A}\n")
print(f"Det(A) = {Det_A}\n")

# calculate eigenvalues and eigenvectors
eig = np.linalg.eig(A)
print(f"Eig: {eig}\n")

# system classification
p = Tr_A ** 2 / 4  # parabola (Tr_A)^2 = 4 Det_A
if Det_A < 0:
  type = "saddle"
elif Tr_A < 0:
  if Det_A < p:
    if Det_A == 0:
      type = "line of stable fixed points"
    else:
      type = "nodal sink"
  elif Det_A > p:
    type = "spiral sink"
  elif Det_A == p:
    type = "degenerate sink"
elif Tr_A > 0:
  if Det_A < p:
    if Det_A == 0:
      type = "line of unstable fixed points"
    else:
      type = "nodal source"
  elif Det_A > p:
    type = "spiral source"
  elif Det_A == p:
    type = "degenerate source"
elif Tr_A == 0:
  if Det_A == 0:
    type = "uniform motion"
  else:
    type = "center"
print(f"{type}\n")

In [ ]:
# @title Prepare Simulation

# simulation parameters
sim_time = 3    # stop time ** max 3 for real eigenvalues
dt = 0.001       # time step
t_span = (0, sim_time)   # for solve_ivp (not used)
t = np.arange(0, sim_time, dt)   # evaluation times
T_R = 0.5        # relaxation period

# store update times
ta = np.array([0])     # alpha
tb = np.array([0])     # beta
tg = np.array([0])     # gamma
td = np.array([0])     # delta

# system parameters
alpha, beta, gamma, delta = A[0,0], A[0,1], A[1,0], A[1,1]
alphat = alpha + 10
betat = beta + 10
gammat = gamma + 10
deltat = delta + 10
At = np.array([[alphat, betat], [gammat, deltat]])

# store parameter estimates
ALPHA = np.array([alphat])
BETA = np.array([betat])
GAMMA = np.array([gammat])
DELTA = np.array([deltat])

# parameter errors
#D_alpha
#D_beta
#D_gamma
#D_delta

# nudging parameters
mu11 = 1000
mu12 = 0
mu21 = 0
mu22 = 1000
M = np.array([[mu11, mu12], [mu21, mu22]])

# initialize system (trajectory 1)
x1_0 = 1
y1_0 = 1
x1t_0 = 5
y1t_0 = 9
X1 = np.array([[x1_0, y1_0]])     # each new solution point will...
X1t = np.array([[x1t_0, y1t_0]])  # ...make a new row
U1 = X1t - X1   # error terms

# initialize system (trajectory 2)
x2_0 = -2
y2_0 = 1
x2t_0 = 2
y2t_0 = 4
X2 = np.array([[x2_0, y2_0]])     # each new solution point will...
X2t = np.array([[x2t_0, y2t_0]])  # ...make a new row
U2 = X2t - X2   # error terms

# solve reference system 1 (forward Euler method)
for i in range(len(t) - 1):
  new_row = X1[i] + (np.matmul(A, X1[i])) * dt
  X1 = np.vstack((X1, new_row))

x1 = X1[:,0]   # column1 = x(t)
y1 = X1[:,1]   # column2 = y(t)

# solve reference system 2 (forward Euler method)
for i in range(len(t) - 1):
  new_row = X2[i] + (np.matmul(A, X2[i])) * dt
  X2 = np.vstack((X2, new_row))

x2 = X2[:,0]   # column1 = x(t)
y2 = X2[:,1]   # column2 = y(t)


In [ ]:
# @title Run Simulation

# run nudging algorithm
for i in range(len(t) - 1):

  if abs(X2t[i,1] * U1[i,0] - X1t[i,1] * U2[i,0]) >= 0 and abs(X2t[i,1] * U1[i,1] - X1t[i,1] * U2[i,1]) >= 0 and X1t[i,0] * X2t[i,1] - X1t[i,1] * X2t[i,0] != 0 and t[i] - ta[-1] >= T_R:
    # update alpha
    At[0,0] = At[0,0] - (mu11 * (X2t[i,1] * U1[i,0] - X1t[i,1] * U2[i,0]) + mu12 * (X2t[i,1] * U1[i,1] - X1t[i,1] * U2[i,1])) / (X1t[i,0] * X2t[i,1] - X1t[i,1] * X2t[i,0])
    # record new estimate
    alpha_est = At[0,0]
    ALPHA = np.append(ALPHA, alpha_est)
    # record update time
    ta = np.append(ta, t[i])

  if abs(X1t[i,0] * U2[i,0] - X2t[i,0] * U1[i,0]) >= 0 and abs(X1t[i,0] * U2[i,1] - X2t[i,0] * U1[i,1]) >= 0 and X1t[i,0] * U2[i,0] - X2t[i,0] * U1[i,0] != 0 and t[i] - tb[-1] >= T_R:
    # update beta
    At[0,1] = At[0,1] - (mu11 * (X1t[i,0] * U2[i,0] - X2t[i,0] * U1[i,0]) + mu12 * (X1t[i,0] * U2[i,1] - X2t[i,0] * U1[i,1])) / (X1t[i,0] * X2t[i,1] - X1t[i,1] * X2t[i,0])
    # record new estimate
    beta_est = At[0,1]
    BETA = np.append(BETA, beta_est)
    # record update time
    tb = np.append(tb, t[i])

  if abs(X2t[i,1] * U1[i,0] - X1t[i,1] * U2[i,0]) >= 0 and abs(X2t[i,1] * U1[i,1] - X1t[i,1] * U2[i,1]) >= 0 and X2t[i,1] * U1[i,1] - X1t[i,1] * U2[i,1] != 0 and t[i] - tg[-1] >= T_R:
    # update gamma
    At[1,0] = At[1,0] - (mu21 * (X2t[i,1] * U1[i,0] - X1t[i,1] * U2[i,0]) + mu22 * (X2t[i,1] * U1[i,1] - X1t[i,1] * U2[i,1])) / (X1t[i,0] * X2t[i,1] - X1t[i,1] * X2t[i,0])
    # record new estimate
    gamma_est = At[1,0]
    GAMMA = np.append(GAMMA, gamma_est)
    # record update time
    tg = np.append(tg, t[i])

  if abs(X1t[i,0] * U2[i,0] - X2t[i,0] * U1[i,0]) >= 0 and abs(X1t[i,0] * U2[i,1] - X2t[i,0] * U1[i,1]) >= 0 and X1t[i,0] * U2[i,1] - X2t[i,0] * U1[i,1] != 0 and t[i] - td[-1] >= T_R:
    # update delta
    At[1,1] = At[1,1] - (mu21 * (X1t[i,0] * U2[i,0] - X2t[i,0] * U1[i,0]) + mu22 * (X1t[i,0] * U2[i,1] - X2t[i,0] * U1[i,1])) / (X1t[i,0] * X2t[i,1] - X1t[i,1] * X2t[i,0])
    # record new estimate
    delta_est = At[1,1]
    DELTA = np.append(DELTA, delta_est)
    # record update time
    td = np.append(td, t[i])

  # integrate auxiliary system 1 (forward Euler method)
  next_row_1 = X1t[i] + (np.matmul(At, X1t[i]) - np.matmul(M, U1[i])) * dt
  X1t = np.vstack((X1t, next_row_1))

  # record new state error 1
  new_err_1 = X1t[i+1] - X1[i+1]
  U1 = np.vstack((U1, new_err_1))

  # integrate auxiliary system 2 (forward Euler method)
  next_row_2 = X2t[i] + (np.matmul(At, X2t[i]) - np.matmul(M, U2[i])) * dt
  X2t = np.vstack((X2t, next_row_2))

  # record new state error 2
  new_err_2 = X2t[i+1] - X2[i+1]
  U2 = np.vstack((U2, new_err_2))

# calculate parameter errors
alpha_err = ALPHA[-1] - alpha
alpha_rel_err = abs(alpha_err / alpha)
beta_err = BETA[-1] - beta
beta_rel_err = abs(beta_err / beta)
gamma_err = GAMMA[-1] - gamma
gamma_rel_err = abs(gamma_err / gamma)
delta_err = DELTA[-1] - delta
delta_rel_err = abs(delta_err / delta)

print(f"X1 = \n {X1}\n")
print(f"X1t = \n {X1t}\n")
print(f"U1[-1] = {U1[-1]}\n")
print(f"X2 = \n {X2}\n")
print(f"X2t = \n {X2t}\n")
print(f"U2[-1] = {U2[-1]}\n")
print(f"alpha = {alpha}\n")
print(f"ALPHA[-1] = {ALPHA[-1]}\n")
print(f"alpha error = {alpha_err}\n")
print(f"alpha rel error = {alpha_rel_err}\n")
print(f"ALPHA = {ALPHA}\n")
print(f"beta = {beta}\n")
print(f"BETA[-1] = {BETA[-1]}\n")
print(f"beta error = {beta_err}\n")
print(f"beta rel error = {beta_rel_err}\n")
print(f"BETA = {BETA}\n")
print(f"gamma = {gamma}\n")
print(f"GAMMA[-1] = {GAMMA[-1]}\n")
print(f"gamma error = {gamma_err}\n")
print(f"gamma rel error = {gamma_rel_err}\n")
print(f"GAMMA = {GAMMA}\n")
print(f"delta = {delta}\n")
print(f"DELTA[-1] = {DELTA[-1]}\n")
print(f"delta error = {delta_err}\n")
print(f"delta rel error = {delta_rel_err}\n")
print(f"DELTA = {DELTA}\n")

print(f"len(t)  = {len(t)}")
print(f"len(ta) = {len(ta)}")
print(f"len(tb) = {len(tb)}")
print(f"len(tg) = {len(tg)}")
print(f"len(td) = {len(td)}\n")

print(f"ta = {ta}")
print(f"tb = {tb}")
print(f"tg = {tg}")
print(f"td = {td}")
